In [1]:
%autosave 0
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))



## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
60,ABW,Aruba,2020-05-22,101,0,3,0,945.994,0.000,28.099,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
194,AFG,Afghanistan,2020-05-22,8676,531,193,6,222.871,13.640,4.958,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
256,AGO,Angola,2020-05-22,58,6,3,0,1.765,0.183,0.091,...,2.405,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-21,France,144352.0,0.0,28263.0,0.0,0.0,0.0
0,2020-05-21,China,84136.0,0.0,4656.0,0.0,0.0,0.0
0,2020-05-21,Italy,228251.0,0.0,32503.0,0.0,0.0,0.0
0,2020-05-21,Spain,232979.0,0.0,27954.0,0.0,0.0,0.0
0,2020-05-21,United States,1573720.0,0.0,95001.0,0.0,0.0,0.0
0,2020-05-21,World,5064423.0,0.0,333454.0,0.0,0.0,0.0
0,2020-05-21,United Kingdom,250729.0,0.0,36206.0,0.0,0.0,0.0
0,2020-05-21,Germany,177581.0,0.0,8223.0,0.0,0.0,0.0
0,2020-05-21,Iran,129088.0,0.0,7251.0,0.0,0.0,0.0
0,2020-05-21,Turkey,153522.0,0.0,4245.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-22,France,144352.0,144163.0,28263.0,28215.0,189.0,48.0
0,2020-05-22,China,84136.0,84079.0,4656.0,4638.0,57.0,18.0
0,2020-05-22,Italy,228251.0,228006.0,32503.0,32486.0,245.0,17.0
0,2020-05-21,Spain,232979.0,233037.0,27954.0,27940.0,-58.0,14.0
0,2020-05-22,United States,1573720.0,1577287.0,95001.0,94702.0,-3567.0,299.0
0,2020-05-22,World,5064423.0,5067579.0,333454.0,332711.0,-3156.0,743.0
0,2020-05-22,United Kingdom,250729.0,250908.0,36206.0,36042.0,-179.0,164.0
0,2020-05-22,Germany,177581.0,177212.0,8223.0,8174.0,369.0,49.0
0,2020-05-22,Iran,129088.0,129341.0,7251.0,7249.0,-253.0,2.0
0,2020-05-22,Turkey,153522.0,153548.0,4245.0,4249.0,-26.0,-4.0


In [7]:
## dump dataset for use it in nb 03, in this nb he will be add to rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")